# Doing inference with SageMaker Built-in Object Detection model

1. [Download the trained model artifact](#download)
1. [Convert training model to deployable model](#convert)
1. [Inference](#inference)
  1. [model load](#load)
  1. [single image inference](#singleinference)
  1. [batch inference](#batchinference)


## setup
### conda_mxnet_p36 is required kernel


In [1]:
#!pip install gluoncv

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import mxnet as mx
import json
import boto3
import os
import cv2

In [3]:
bucket='deeplens-imageclassification-matthew'
endEpoch=60
TMP_FOLDER = 'trained-model'

When running this script, you need to make sure that command line options you pass in match exactly the hyperparameters of your training job. If you’re unsure, refer the hyperparams.json file in your unpacked model artifacts to confirm. 

Save the copy of the deployable model artifact in S3

# 3. Doing inference with the model on local host <a id='inference'></a>

Below code will run inference on a set of test images on the current notebook instance. Using a GPU instance (e.g. p2.\*, p3.\* family) will result in faster performance than CPU only instances. You can stop the SageMaker notebook instance and update the instance type, and restart the notebook instance before continuing. 

In [4]:
def get_ctx():
    try:
        gpus = mx.test_utils.list_gpus()
        if len(gpus) > 0:
            ctx = []
            for gpu in gpus:
                ctx.append(mx.gpu(gpu))
        else:
            ctx = [mx.cpu()]
    except:
        ctx = [mx.cpu()]
    return ctx

ctx = get_ctx()[0]
ctx

cpu(0)

In [5]:
SHAPE = 224
input_shapes=[('data', (1, 3, SHAPE, SHAPE))]
confidence_threshold = 0.3



## 3A. Loading the model <a id="load"> </a>

In [6]:
%%time

param_path=os.path.join(TMP_FOLDER, 'image-classification')
print("param_path: {}".format(param_path))
sym, arg_params, aux_params = mx.model.load_checkpoint(param_path, endEpoch)#matthew : 60 is the latest epoch
#print(arg_params)
#print(sym)
#print(aux_params)
print(100)
mod = mx.mod.Module(symbol=sym, label_names=[], context = ctx)
print(200)
mod.bind(for_training=False, data_shapes=input_shapes, label_shapes = None)
print(300)
# Create module
#model = mx.mod.Module(symbol = mx_sym, context = mx.cpu(), label_names=[])

# Bind the data shape and load params
#model.bind(for_training=False,
#           data_shapes = [('data',(1,3,224,224))],
#           label_shapes = None))
#model.set_params(args, auxs)

mod.set_params(arg_params, aux_params, allow_missing=True)

param_path: trained-model/image-classification
100
200
300
CPU times: user 122 ms, sys: 80.9 ms, total: 203 ms
Wall time: 154 ms


In [7]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

def predict_from_file(filepath, reshape=(SHAPE, SHAPE)):
    # Switch RGB to BGR format (which ImageNet networks take)
    img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
    if img is None:
        return []

     # Resize image to fit network input
    img = cv2.resize(img, reshape)
    
    org_image = img.copy()
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
 
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    prob = np.squeeze(prob)

    return prob, org_image

In [8]:
def infer(image_path, threshold=confidence_threshold):
    results, org_image = predict_from_file(image_path)
    image_name = image_path.split("/")[-1]
    print(results)
    maxId=np.argmax(results)
    strDet=''
    if maxId==0 and results[maxId] > 0.9:
        strDet='cat'
    elif maxId==1 and results[maxId] > 0.9 :
        strDet='dog'  
    else :
        strDet='other'

    
    return strDet, results[maxId]

## 3B. Test inference on single image <a id="singleinference"></a>

In [9]:
%%time

prediction_image = 'img/Taka_Shiba.jpg'
#prediction_image = 'img/1280px-Danish_bicycle_female.jpg'
prediction_image = 'img/img_1317.jpg'
#prediction_image = 'img/cat.png'
label, confidence = infer(prediction_image)
print(label)
print(confidence)

[9.9991298e-01 8.6768858e-05 2.8140613e-07]
cat
0.999913
CPU times: user 384 ms, sys: 136 ms, total: 520 ms
Wall time: 434 ms
